In [1]:
from datetime import date
import tensorflow as tf
import numpy as np

2024-04-04 10:21:55.768768: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
MONTHS = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]


def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.choice(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[date.month - 1] + " " + date.strftime("%d, %Y") for date in dates]
    y = [date.isoformat() for date in dates]
    return x, y

In [3]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


Let's get the list of all possible characters in the inputs.

In [4]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

And also the list of all possible characters in the outputs. 

In [5]:
OUTPUT_CHARS = "0123456789-"

Let's write a function to convert a string to a list of character IDs, as we did in the previous exercise.

In [6]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [7]:
date_str_to_ids(x_example[0], INPUT_CHARS)

[19, 23, 31, 34, 23, 28, 21, 23, 32, 0, 4, 2, 1, 0, 9, 2, 9, 7]

In [8]:
date_str_to_ids(y_example[0], OUTPUT_CHARS)

[7, 0, 7, 5, 10, 0, 9, 10, 2, 0]

In [9]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor()  # using 0 as the padding token ID


def create_datasets(n_dates):
    X, y = random_dates(n_dates)
    return prepare_date_strs(X, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [10]:
np.random.seed(42)

X_train, y_train = create_datasets(10_000)
X_valid, y_valid = create_datasets(2_000)
X_test, y_test = create_datasets(2_000)

2024-04-04 10:21:59.039191: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1], dtype=int32)>

### First version: a very basic seq2seq model

Let's first try the simplest model: we feed in the input sequence, which first goes through the encoder (an embedding layer followed by a single LSTM layer), which outputs a vector, then it goes through a decoder (a single LSTM layer), which outputs a sequence of vectors, each representing the estimated probabilities for all possible output characters.

Since the decoder expects a sequence as input, we repeat the vector (which is output by the encoder) as many times as the longest possible output sequence.

In [12]:
embedding_size = 32
max_output_length = y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(
            input_dim=len(INPUT_CHARS) + 1,
            output_dim=embedding_size,
            input_shape=[None],
        ),
        tf.keras.layers.LSTM(128),
    ]
)

decoder = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(128, return_sequences=True),
        tf.keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax"),
    ]
)

model = tf.keras.Sequential(
    [encoder, tf.keras.layers.RepeatVector(max_output_length), decoder]
)

optimizer = tf.keras.optimizers.Nadam()
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

2024-04-04 10:22:00.434079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 10:22:00.435233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 10:22:00.435887: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/20


2024-04-04 10:22:01.014155: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 10:22:01.015308: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 10:22:01.017026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

312/313 [============================>.] - ETA: 0s - loss: 1.6899 - accuracy: 0.3970

2024-04-04 10:22:15.695092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 10:22:15.696920: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 10:22:15.698035: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

313/313 [==============================] - 16s 42ms/step - loss: 1.6892 - accuracy: 0.3972 - val_loss: 1.2365 - val_accuracy: 0.5601
Epoch 2/20
313/313 [==============================] - 13s 43ms/step - loss: 1.2915 - accuracy: 0.5538 - val_loss: 1.0024 - val_accuracy: 0.6407
Epoch 3/20
313/313 [==============================] - 12s 40ms/step - loss: 0.8937 - accuracy: 0.6739 - val_loss: 0.7667 - val_accuracy: 0.7082
Epoch 4/20
313/313 [==============================] - 17s 53ms/step - loss: 0.6565 - accuracy: 0.7421 - val_loss: 0.5791 - val_accuracy: 0.7691
Epoch 5/20
313/313 [==============================] - 16s 52ms/step - loss: 0.4791 - accuracy: 0.8090 - val_loss: 0.4062 - val_accuracy: 0.8392
Epoch 6/20
313/313 [==============================] - 13s 42ms/step - loss: 0.3191 - accuracy: 0.8801 - val_loss: 0.2563 - val_accuracy: 0.9126
Epoch 7/20
313/313 [==============================] - 10s 31ms/step - loss: 0.2750 - accuracy: 0.9144 - val_loss: 0.1960 - val_accuracy: 0.9420
Epo

Looks great, we reach 100% validation accuracy! Let's use the model to make some predictions. We will need to be able to convert a sequence of character IDs to a readable string.

In [13]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS) -> list:
    return ["".join([("?" + chars)[index] for index in sequence]) for sequence in ids]

In [14]:
X_new = prepare_date_strs(["September 17, 2009", "July 14, 1789"])

In [15]:
ids = model.predict(X_new, verbose=0).argmax(axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2024-04-04 10:26:39.704912: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 10:26:39.706729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 10:26:39.707931: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2009-09-17
1789-07-14


Perfect! However, since the model was trained on input strings of length 18 (which is the length of the longest date and since we padded all the input sequences), it does not perform well if we try to use it to make predictions on shorter sequences.

In [16]:
X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"])

In [17]:
ids = model.predict(X_new, verbose=0).argmax(axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

2024-04-04 10:26:40.460959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 10:26:40.462310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 10:26:40.463507: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2020-01-02
1789-09-14


Okay, we need to ensure that we always pass sequences of the same length as during training, using padding if necessary.

In [18]:
max_input_length = X_train.shape[1]


def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X


def convert_date_strs(date_str):
    X = prepare_date_strs_padded(date_str)
    ids = model.predict(X, verbose=0).argmax(axis=-1)
    return ids_to_date_strs(ids)

In [19]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

['2020-05-02', '1789-07-14']

Of course, there are much easier way to write a conversion date tool (e.g., using regular expressions or even basic string manipulation), but yeah, using neural network is way much cooler.

However, real-life sequence-to-sequence problems are usually much harder, so for the sake of completeness, we will try to build a more powerful model.

### Second version: feeding the shifted targets to the decoder (teacher forcing)

- Instead of feeding the decoder a simple repetition of the encoder's output vector, we can feed it the target sequence, shifted by one time step to the right.
- This way, at each time step, the decoder will know what the previous target character was. This should help to tackle more complex sequence-to-sequence problems.
- Since the first output character of each target sequences has no previous character, we will need a new token to represent the start-of-sequence (sos).
- During inference, we won't know the target, so what will we feed the decoder? We can just predict one character at a time, starting with an sos token, and feeding the decoder all the characters that were predicted so far.
- But if the decoder's LSTM expects to egt the previous target as input at each step, how would we pass i the output vector by the encoder? Well, we can ignore the output vector, and instead use the encoder's LSTM state as the initial state of the decoder LSTM (which requires that the encoder's LSTM must have the same number of units as the decoder's LSTM).
- Now, let's create the decoder's inputs (for training, validation and testing).
- The sos token wil represented using the last possible output character's ID + 1.

In [22]:
sos_id = len(OUTPUT_CHARS) + 1


def shifted_output_sequence(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)


X_train_decoder = shifted_output_sequence(y_train)
X_valid_decoder = shifted_output_sequence(y_valid)
X_test_decoder = shifted_output_sequence(y_test)

Let's have a look at the decoder's training inputs.

In [23]:
X_train_decoder

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[12,  8,  1, ..., 10, 11,  3],
       [12,  9,  6, ...,  6, 11,  2],
       [12,  8,  2, ...,  2, 11,  2],
       ...,
       [12, 10,  8, ...,  2, 11,  4],
       [12,  2,  2, ...,  3, 11,  3],
       [12,  8,  9, ...,  8, 11,  3]], dtype=int32)>

Now, let's build the model. It's not a simple sequential model anymore, so we'll use the functional API.

In [28]:
encoder_embed_size = 32
decoder_embed_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = tf.keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = tf.keras.layers.Embedding(
    input_dim=len(INPUT_CHARS) + 1, output_dim=encoder_embed_size
)(encoder_input)
_, encoder_state_h, encoder_state_c = tf.keras.layers.LSTM(
    lstm_units, return_state=True
)(encoder_embedding)
encoder_state = [encoder_state_c, encoder_state_h]

decoder_input = tf.keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = tf.keras.layers.Embedding(
    input_dim=len(OUTPUT_CHARS) + 2, output_dim=decoder_embed_size
)(decoder_input)
decoder_output = tf.keras.layers.LSTM(lstm_units, return_sequences=True)(
    decoder_embedding, initial_state=encoder_state
)
output = tf.keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")(
    decoder_output
)

model = tf.keras.Model(inputs=[encoder_input, decoder_input], outputs=[output])

2024-04-04 11:47:27.323596: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 11:47:27.324702: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 11:47:27.325369: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [29]:
optimizer = tf.keras.optimizers.Nadam()
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)
model.fit(
    (X_train, X_train_decoder),
    y_train,
    epochs=10,
    validation_data=((X_valid, X_valid_decoder), y_valid),
)

Epoch 1/10


2024-04-04 11:47:28.868830: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 11:47:28.869937: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 11:47:28.870660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

312/313 [============================>.] - ETA: 0s - loss: 1.6725 - accuracy: 0.3744

2024-04-04 11:47:44.239396: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 11:47:44.240342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 11:47:44.241321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

313/313 [==============================] - 17s 45ms/step - loss: 1.6720 - accuracy: 0.3745 - val_loss: 1.3579 - val_accuracy: 0.4771
Epoch 2/10
313/313 [==============================] - 12s 40ms/step - loss: 1.0972 - accuracy: 0.5921 - val_loss: 0.7779 - val_accuracy: 0.7223
Epoch 3/10
313/313 [==============================] - 13s 40ms/step - loss: 0.5524 - accuracy: 0.8099 - val_loss: 0.3492 - val_accuracy: 0.8933
Epoch 4/10
313/313 [==============================] - 11s 36ms/step - loss: 0.2277 - accuracy: 0.9411 - val_loss: 0.1578 - val_accuracy: 0.9661
Epoch 5/10
313/313 [==============================] - 13s 41ms/step - loss: 0.1095 - accuracy: 0.9805 - val_loss: 0.0727 - val_accuracy: 0.9902
Epoch 6/10
313/313 [==============================] - 13s 40ms/step - loss: 0.0469 - accuracy: 0.9961 - val_loss: 0.0433 - val_accuracy: 0.9951
Epoch 7/10
313/313 [==============================] - 15s 48ms/step - loss: 0.0280 - accuracy: 0.9985 - val_loss: 0.0284 - val_accuracy: 0.9977
Epo

This model also reaches nearly 100% validation accuracy, but is does so much faster.

In [30]:
sos_id = len(OUTPUT_CHARS) + 1


def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    y_predict = tf.fill(dims=(len(X), 1), value=sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - y_predict.shape[1]
        X_decoder = tf.pad(y_predict, [[0, 0], [0, pad_size]])
        y_proba_next = model.predict([X, X_decoder], verbose=0)[:, index : index + 1]
        y_predict_next = tf.argmax(y_proba_next, axis=-1, output_type=tf.int32)
        y_predict = tf.concat([y_predict, y_predict_next], axis=1)
    return ids_to_date_strs(y_predict[:, 1:])

In [31]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

2024-04-04 12:03:27.404422: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-04 12:03:27.405194: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-04 12:03:27.406315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

['1789-07-14', '2020-05-01']

You can try to write a Transformer version.